In [20]:
from __future__ import print_function

import tensorflow as tf

def fgsm(x, predictions, eps):
    # Get Loss
    y = tf.to_float(tf.equal(predictions, tf.reduce_max(predictions, 1, keep_dims=True)))
    y = y / tf.reduce_sum(y, 1, keep_dims=True)
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y)
    
    # Define gradient of loss with respect to input
    grad = tf.gradients(loss, x)[0]
    
    # Take sign of gradient
    signed_grad = tf.sign(grad)

    # Multiply by constant epsilon
    scaled_signed_grad = eps * signed_grad

    # Add perturbation to original example to obtain adversarial example
    adv_x = tf.stop_gradient(x + scaled_signed_grad)
    return adv_x


# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# CNN Model

# Input Data and Labels
training_data = tf.placeholder(tf.float32, [None, 784])
training_labels = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

# Defining Variables
layer1 = tf.Variable(tf.random_normal([5,5,1,32]))
bias1 = tf.Variable(tf.random_normal([32]))

# 1st Convoluted Layer
x = tf.reshape(training_data, shape = [-1, 28, 28, 1])
x = tf.nn.conv2d(x, layer1, strides = [1,1,1,1], padding = 'SAME')
x = tf.nn.bias_add(x, bias1)
x = tf.nn.relu(x)

#1st Pooling Layer
x = tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

# Defining Variables
layer2 = tf.Variable(tf.random_normal([5,5,32,64]))
bias2 = tf.Variable(tf.random_normal([64]))

# 2nd Convoluted Layer
x = tf.nn.conv2d(x, layer2, strides =[1,1,1,1], padding = 'SAME')
x = tf.nn.bias_add(x, bias2)
x = tf.nn.relu(x)

#2nd Pooling Layer
x = tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

#Defining Variables
layer3 = tf.Variable(tf.random_normal([7*7*64, 1024]))
bias3 = tf.Variable(tf.random_normal([1024]))

#Fully-Connected Layer
x = tf.reshape(x, [-1, 7*7*64])
x = tf.add(tf.matmul(x, layer3), bias3)
x = tf.nn.relu(x)

x = tf.nn.dropout(x, keep_prob)

#Converting to class scores
layer4 = tf.Variable(tf.random_normal([1024, 10]))
bias4 = tf.Variable(tf.random_normal([10]))

output =  tf.add(tf.matmul(x, layer4), bias4)
fgsm = fgsm(training_data, output, 0.01)
# Calculating Loss Function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=training_labels))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

correct_pred = tf.equal(tf.argmax(output, 1), tf.argmax(training_labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()


with tf.Session() as sess:
    sess.run(init)
    step = 1
    
    # Keep training until reach max iterations
    while step * 128 < 200000:
        batch_x, batch_y = mnist.train.next_batch(128)
        # Run optimization op (backprop)
        opt, fg =sess.run([optimizer, fgsm], feed_dict = {training_data: batch_x, training_labels: batch_y, keep_prob: 0.75})
        
        sess.run(optimizer, feed_dict = {training_data: fg, training_labels: batch_y, keep_prob: 0.75})
        if step % 10 == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={training_data: batch_x,
                                                            training_labels: batch_y, keep_prob: 1
                                                              })
            print("Iter " + str(step*128) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    
print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={training_data: mnist.test.images[:256],
                                      training_labels: mnist.test.labels[:256],
                                      keep_prob: 1.}))
    

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Iter 1280, Minibatch Loss= 9166.421875, Training Accuracy= 0.60938
Iter 2560, Minibatch Loss= 3657.925537, Training Accuracy= 0.74219
Iter 3840, Minibatch Loss= 4461.915527, Training Accuracy= 0.76562
Iter 5120, Minibatch Loss= 3078.172607, Training Accuracy= 0.85938
Iter 6400, Minibatch Loss= 2064.656494, Training Accuracy= 0.85938
Iter 7680, Minibatch Loss= 3662.253662, Training Accuracy= 0.83594
Iter 8960, Minibatch Loss= 1539.263672, Training Accuracy= 0.88281
Iter 10240, Minibatch Loss= 1875.014526, Training Accuracy= 0.91406
Iter 11520, Minibatch Loss= 851.039307, Training Accuracy= 0.96094
Iter 12800, Minibatch Loss= 1848.076904, Training Accuracy= 0.87500
Iter 14080, Minibatch Loss= 438.071960, Training Accuracy= 0.95312
Iter 15360, Minibatch Loss= 534.241638, Training Accuracy= 0.96875
Iter

RuntimeError: Attempted to use a closed Session.